In [14]:
import pandas as pd
import numpy as np
import values
import uuid
import random
from datetime import datetime, timedelta

##### Tables

In [15]:
# CDHDR
cdhdr_cols = ['MANDT', 'OBJECTCLAS', 'OBJECTID', 'CHANGENR', 'USERNAME', 'ERDAT']
cdhdr = pd.DataFrame(columns=cdhdr_cols)

# CDPOS
cdpos_cols = ['MANDT', 'CHANGENR', 'TABNAME', 'FNAME', 'OLDVALUE', 'NEWVALUE']
cdpos = pd.DataFrame(columns=cdpos_cols)

# Materials
mara_cols = ['MANDT', 'MATNR', 'MAKTX']
mara = pd.DataFrame(columns=mara_cols)

# Users
usr01_cols = ['MANDT', 'BNAME', 'UTYPE']
usr01 = pd.DataFrame(columns=usr01_cols)

# Customers
Kna1_cols = ['MANDT', 'KUNNR', 'NAME1', 'ERDAT', 'ERNAM']
kna1 = pd.DataFrame(columns=Kna1_cols)

# Sales Order header
vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'VSBED']
vbak = pd.DataFrame(columns=vbak_cols)

# Sales Order items
vbap_cols = ['MANDT', 'VBELN', 'POSNR', 'KWMENG', 'MATNR', 'NETWR']
vbap =pd.DataFrame(columns=vbap_cols)

# Delivery header
likp_cols = ['MANDT', 'VBELN', 'WADAT', 'LFUHR', 'ERDAT', 'ERNAM']
likp = pd.DataFrame(columns=likp_cols)

# Delivery items
lips_cols = ['MANDT', 'VBELN', 'POSNR']
lips = pd.DataFrame(columns=lips_cols)

# Header status
vbuk_cols = ['MANDT', 'VBELN', 'LFSTK', 'FKSTK', 'AEDAT']
vbuk = pd.DataFrame(columns=vbuk_cols)

# Material movement header
mkpf_cols = ['MANDT', 'MBLNR', 'BLART', 'AEDAT', 'USNAM']
mkpf = pd.DataFrame(columns=mkpf_cols)

# Material movement items
mseg_cols = ['MANDT', 'MBLNR', 'KDAUF', 'KDPOS']
mseg = pd.DataFrame(columns=mseg_cols)

# Billing header
vbrk_cols = ['MANDT', 'VBELN', 'VBTYP', 'ERNAM', 'ERDAT']
vbrk  =pd.DataFrame(columns=vbrk_cols)

# Billing items
vbrp_cols = ['MANDT', 'VBELN', 'POSNR']
vbrp  =pd.DataFrame(columns=vbrp_cols)

##### Helpers

In [16]:
vbak_len = 100

In [17]:
def generate_random_datetime(start_date, end_date):
    time_delta = end_date - start_date
    random_days = random.randint(0, time_delta.days)
    random_seconds = random.randint(0, 86400)  # 86400 seconds in a day

    random_datetime = start_date + timedelta(days=random_days, seconds=random_seconds)
    return random_datetime

def add_random_datetime(start_time: datetime):
    random_days = random.randint(1, 365)
    random_hours = random.randint(0, 23)
    random_minutes = random.randint(0, 59)
    new_dt = start_time + timedelta(days=random_days, hours=random_hours, minutes=random_minutes)

    return new_dt

def divide_num_in_normal(num: float, num_parts: int):
    mean = num/2
    std_dev = num/15

    random_numbers = np.random.normal(mean, std_dev, num_parts)
    random_numbers /= sum(random_numbers)

    result = [int(number * num) for number in random_numbers]
    result[-1] += num - sum(result)

    return result

In [18]:
# Materials
mara_temp_list = []
for i in range(100):
    rand_id = uuid.uuid4()
    mara_temp_list.append([values.mandt, rand_id, values.material_names[i]])

mara = pd.concat([mara, pd.DataFrame(mara_temp_list, columns=mara_cols)], ignore_index=True)

In [19]:
# Users
usr01_temp_list = []
for u_name, u_type in values.users.items():
    usr01_temp_list.append([values.mandt, u_name, u_type])

usr01 = pd.concat([usr01, pd.DataFrame(usr01_temp_list, columns=usr01_cols)], ignore_index=True)

In [20]:
# Customers
kna1_temp_list = []
for company in values.company_names:
    rand_id = uuid.uuid4()
    rand_dt = add_random_datetime(start_time=values.times['customer_create_start'])
    rand_usr = random.choice(list(values.users.keys()))
    kna1_temp_list.append([values.mandt, rand_id, company, rand_dt, rand_usr])

kna1 = pd.concat([kna1, pd.DataFrame(kna1_temp_list, columns=Kna1_cols)], ignore_index=True)

In [21]:
vbak_temp_list = []
vbap_temp_list = []
likp_temp_list = []
lips_temp_list = []
cdhdr_changes_temp_list = []
cdpos_changes_temp_list = []
vbuk_temp_list = []
mkpf_temp_list = []
mseg_temp_list = []
vbrk_temp_list = []
vbrp_temp_list = []

In [22]:
class SalesOrderItem:
    def __init__(self, vbeln, posnr, kwmeng, matnr, netwr) -> None:
        self.mandt = values.mandt
        self.vbeln = vbeln
        self.posnr = posnr
        self.kwmeng = kwmeng
        self.matnr = matnr
        self.netwr = netwr
        
class SalesOrder:
    def __init__(self) -> None:
        global vbak_temp_list
        global vbap_temp_list
        global likp_temp_list
        global lips_temp_list
        global cdhdr_changes_temp_list
        global cdpos_changes_temp_list
        global mkpf_temp_list
        global mseg_temp_list
        global vbrk_temp_list
        global vbrp_temp_list
        global kna1

        self.mandt = values.mandt
        self.vbeln = uuid.uuid4()
        rand_kna1 = kna1.sample(n=1)
        self.erdat = generate_random_datetime(datetime.fromtimestamp(rand_kna1['ERDAT'].values[0].tolist() / 1e9), values.times['salesorder_create_end'])
        self.ernam = random.choice(list(values.users.keys()))
        self.kunnr = rand_kna1['KUNNR'].values[0]
        self.netwr = round(random.uniform(500, 15_000), 2)
        self.vsbed = random.choice(values.shipping_conditions)

        self.vbaps = []
        self.vbuk = None

        self.delivery_doc_created_at = None
        self.planned_delivery_dt = None
        self.actual_delivery_dt = None
        self.delivery_released_at = None
        self.goods_shipped_at = None
        self.delivery_confirmed_at = None
        self.invoice_sent_at = None
        self.invoice_cleared_at = None
        
        # Create Sales order 
        vbak_temp_list.append([values.mandt, self.vbeln, self.erdat, self.ernam, self.kunnr, self.netwr, self.vsbed])
        self.create_vbap_rows(num_items=(random.randint(5, 15)))


    def create_vbap_rows(self, num_items):
        # Activity: Create sales order - item
        vbap_netwrs = divide_num_in_normal(num=self.netwr, num_parts=num_items)
        
        for j in range(num_items):
            rand_matnr = mara.sample(n=1)['MATNR'].values[0]
            quantity = random.randint(50, 150)
            vbap_temp_list.append([values.mandt, self.vbeln, j, quantity, rand_matnr, vbap_netwrs[j]])
            self.vbaps.append(SalesOrderItem(self.vbeln, j, quantity, rand_matnr, vbap_netwrs[j]))

    def generate_delivery_doc(self):
        # Activity: Generate delivery document
        self.delivery_doc_created_at = add_random_datetime(start_time=self.erdat)
        self.planned_delivery_dt = add_random_datetime(start_time=self.delivery_doc_created_at)
        self.actual_delivery_dt = self.planned_delivery_dt if random.uniform(0, 1) > 0.25 else add_random_datetime(start_time=self.planned_delivery_dt)
        rand_usr = random.choice(list(values.users.keys()))
        likp_temp_list.append([values.mandt, self.vbeln, self.planned_delivery_dt, self.actual_delivery_dt, self.delivery_doc_created_at, rand_usr])
        
        self.generate_delivery_doc_item()
    
    def generate_delivery_doc_item(self):
        # Activity: Generate delivery document - item
        for vbap_elem in self.vbaps:
            lips_temp_list.append([values.mandt, self.vbeln, vbap_elem.posnr])

    def release_delivery(self):
        # Activity: Release delivery
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))
        self.delivery_released_at = add_random_datetime(start_time=self.delivery_doc_created_at)

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'DELIVERY', self.vbeln, rand_changenr, rand_usr, self.delivery_released_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'LFSTK', pd.NA, 'DELIVERYRELEASED'])
        
        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYRELEASED', pd.NA, self.delivery_released_at]

    def ship_goods(self):
        # Activity: Ship goods
        rand_mblnr = uuid.uuid4()
        self.goods_shipped_at = add_random_datetime(start_time=self.delivery_released_at)
        rand_usr = random.choice(list(values.users.keys()))

        mkpf_temp_list.append([values.mandt, rand_mblnr, 'GOODSISSUE', self.goods_shipped_at, rand_usr])

        for vbap_elem in self.vbaps:
            mseg_temp_list.append([values.mandt, rand_mblnr, self.vbeln, vbap_elem.posnr])

    def send_invoice(self):
        # Activity: Send invoice
        self.invoice_sent_at =  add_random_datetime(start_time=self.goods_shipped_at)
        rand_usr = random.choice(list(values.users.keys()))

        vbrk_temp_list.append([values.mandt, self.vbeln, 'INVOICE', rand_usr, self.invoice_sent_at])
        
        self.send_invoice_items()

    def send_invoice_items(self):
        for vbap_elem in self.vbaps:
            vbrp_temp_list.append([values.mandt, self.vbeln, vbap_elem.posnr])
    
    def receive_delivery_confirmation(self):
        # Activity: Recieve delivery confirmation
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.delivery_confirmed_at = add_random_datetime(start_time=self.invoice_sent_at)

        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYCONFIRMED', pd.NA, self.delivery_confirmed_at]
        # vbuk.loc[i, 'LFSTK'] = 'DELIVERYCONFIRMED'
        # vbuk.loc[i, 'ERDAT'] = self.delivery_confirmed_at

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'DELIVERY', self.vbeln, rand_changenr, rand_usr, self.delivery_confirmed_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'LFSTK', 'DELIVERYRELEASED', 'DELIVERYCONFIRMED'])

    def clear_invoice(self):
        # Activity: Clear invoice
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.invoice_cleared_at = add_random_datetime(start_time=self.delivery_confirmed_at)

        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYCONFIRMED', 'INVOICECLEARED', self.invoice_cleared_at]
        # vbuk.loc[i, 'FKSTK'] = 'INVOICECLEARED'
        # vbuk.loc[i, 'ERDAT'] = self.invoice_cleared_at

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbeln, rand_changenr, rand_usr, self.invoice_cleared_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'FKSTK', pd.NA, 'INVOICECLEARED'])


In [23]:
for i in range(5):
    vbak_obj = SalesOrder()
    vbak_obj.generate_delivery_doc()
    vbak_obj.release_delivery()
    vbak_obj.ship_goods()
    vbak_obj.send_invoice()
    vbak_obj.receive_delivery_confirmation()
    vbak_obj.clear_invoice()
    vbuk_temp_list.append(vbak_obj.vbuk)

In [26]:
vbak = pd.concat([vbak, pd.DataFrame(vbak_temp_list, columns=vbak_cols)], ignore_index=True)
vbap = pd.concat([vbap, pd.DataFrame(vbap_temp_list, columns=vbap_cols)], ignore_index=True)
likp = pd.concat([likp, pd.DataFrame(likp_temp_list, columns=likp_cols)], ignore_index=True)
lips = pd.concat([lips, pd.DataFrame(lips_temp_list, columns=lips_cols)], ignore_index=True)
cdhdr = pd.concat([cdhdr, pd.DataFrame(cdhdr_changes_temp_list, columns=cdhdr_cols)], ignore_index=True)
cdpos = pd.concat([cdpos, pd.DataFrame(cdpos_changes_temp_list, columns=cdpos_cols)], ignore_index=True)
vbuk = pd.concat([vbuk, pd.DataFrame(vbuk_temp_list, columns=vbuk_cols)], ignore_index=True)
mkpf = pd.concat([mkpf, pd.DataFrame(mkpf_temp_list, columns=mkpf_cols)], ignore_index=True)
mseg = pd.concat([mseg, pd.DataFrame(mseg_temp_list, columns=mseg_cols)], ignore_index=True)
vbrk = pd.concat([vbrk, pd.DataFrame(vbrk_temp_list, columns=vbrk_cols)], ignore_index=True)
vbrp = pd.concat([vbrp, pd.DataFrame(vbrp_temp_list, columns=vbrp_cols)], ignore_index=True)

##### Save csv

In [27]:
cdhdr.to_csv(f'data/cdhdr.csv', index=False) 
cdpos.to_csv(f'data/cdpos.csv', index=False) 
mara.to_csv(f'data/mara.csv', index=False) 
usr01.to_csv(f'data/usr01.csv', index=False) 
kna1.to_csv(f'data/kna1.csv', index=False) 
vbak.to_csv(f'data/vbak.csv', index=False) 
vbap.to_csv(f'data/vbap.csv', index=False) 
likp.to_csv(f'data/likp.csv', index=False) 
lips.to_csv(f'data/lips.csv', index=False) 
vbuk.to_csv(f'data/vbuk.csv', index=False) 
mkpf.to_csv(f'data/mkpf.csv', index=False) 
mseg.to_csv(f'data/mseg.csv', index=False) 
vbrk .to_csv(f'data/vbrk.csv', index=False) 
vbrp .to_csv(f'data/vbrp.csv', index=False) 